## Lista kroków "manualnych"
1. Wejdź na stronę produktu/z opiniami o produkcie.
2. Dla każdej opinii na stronie
    1. Skopiuj opinię
    2. Wklej opinię do edytora tekstu
3. Przejdź do kolejnej strony z opiniami
4. Powtarzaj kroki 2-3 dopóki są strony z opiniami

## Lista kroków scrapera
1. Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom.
3. Dla każdej opinii na stronie
    1. Wydobycie z kodu opinii jej składowych
    2. Dodanie słownika reprezentującego pojedynczą opinię do listy


## Struktura opinii w serwisie Ceneo.pl
|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autor|author|span.user-post__author-name|
|rekomendacja|recommendation|span.user-post__recomendation > em|
|liczba gwiazdek|stars|span.user-post__score-count|
|treść opinii|content|div.user-post__text|
|lista zalet|pros|div.review-feature__title--positives ~ div.review-feature__item|
|lista wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|data wystawienia opinii|post_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|useful|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|useless|button.vote-no > span|

In [2]:
149845826
158186036
152295133
162688719
162846924

162846924

1. Import bibliotek.

In [4]:
import requests
from bs4 import BeautifulSoup
import os
import json

2. Wysłanie ządania dostępu do zasobu.

In [5]:
product_code = "149845826"
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)
print(response.status_code)

200


3. Wydobycie z kodu HTML fragmentów  odpowiadających opiniom

In [6]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")

4. Wydobycie z kodu opinii jej składowych dla każdej opinii na stronie

In [12]:
all_opinions = []
for opinion in opinions:
    single_opinion = {
        'opinion_id': opinion["data-entry-id"].strip(),
        'author': opinion.select_one("span.user-post__author-name").text.strip() if opinion.select_one("span.user-post__author-name") else '',
        'recommendation': opinion.select_one("span.user-post__recomendation > em").text.strip() if opinion.select_one("span.user-post__recomendation > em") else '',
        'stars': opinion.select_one("span.user-post__score-count").text.strip() if opinion.select_one("span.user-post__score-count") else '',
        'content': opinion.select_one("div.user-post__text").text.strip() if opinion.select_one("div.user-post__text") else '',
        'pros': [p.text.strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
        'cons': [c.text.strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
        'post_date': opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip() if opinion.select_one("span.user-post__published > time:nth-child(1)") else '',
        # 'purchase_date': opinion.select_one("span.user-post__published > time:nth-child(2)").get("datetime", "").strip() if opinion.select_one("span.user-post__published > time:nth-child(2)") else '',
        'useful': opinion.select_one("button.vote-yes > span").text.strip() if opinion.select_one("button.vote-yes > span") else '',
        'useless': opinion.select_one("button.vote-no > span").text.strip() if opinion.select_one("button.vote-no > span") else '',
    }
    all_opinions.append(single_opinion)

print(all_opinions)


[{'opinion_id': '17644074', 'author': 'h...z', 'recommendation': '', 'stars': '5/5', 'content': '#Promocja-Galaxy \nTelefon jest szybki, działa bez zacięć. Robi dobre zdjęcia, bateria trzyma dwa dni,\nbardzo dobry głośnik. Dużym plusem jest że ma miejsce na kartę microSD.\nBardzo dobry smartfon. Jestem bardzo zadowolona z zakupu.', 'pros': [], 'cons': [], 'post_date': '2023-06-24 10:14:32', 'useful': '6', 'useless': '2'}, {'opinion_id': '17497612', 'author': 'M...k', 'recommendation': '', 'stars': '4,5/5', 'content': 'Przesiadłam się z Samsunga A50 na A54 więc przeskok jest spory. Nie lubię zbyt często zmieniać telefonu więc pomyślałam że poprostu przesiądę się na nowszy model. A53 mnie nie przekonało więc czekałam na A54. Po miesiącu użytkowania mogę powiedzieć że:\n*robi fajne zdjęcia, \n*bluetooth szybko paruje się z urzadzeniami, \n*jakość rozmów jest dobra,\n*aplikacje socjal mediów się nie zacinają,\n*szybko łączy się z internetem czy to WiFi czy transmisja danych,\n*bateria trzy

5. Zapis opinii do pliku JSON

In [28]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_code}.json", "w", encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()